# MANUAL SKETCHBOOK

## PREPARACIÓN 

### Python 3.X

https://www.python.org/downloads/

### Librerias necesarias

- selenium
- utm
- requests
- pandas
- numpy

### Chrome-driver

https://chromedriver.storage.googleapis.com/index.html?path=89.0.4389.23/

## ORGANIZACIÓN DE LA BASE DE DATOS 

La realización de esto se hará en base al siguiente documento https://docs.gbif-uat.org/georeferencing-quick-reference-guide/1.0/en/georeferencing-quick-reference-guide-community-review-draft.en.pdf (punto 1.5)

En resumen, tendremos dos posibles casos de georreferenciación.

El primero corresponde a la existencia de un punto con coordenadas geograficas definidas, por lo que es necesario rellenar los siguientes campos
- verbatimLatitude
- verbatimLongitude
- verbatimSRS


## PROCESAMIENTO DE LA BASE DE DATOS

## AUTOMATIZACIÓN DE "GEOREFERENCING CALCULATOR"

## POST PROCESAMIENTO DE LA BASE DE DATOS

## EXPORTAR BASE DE DATOS

In [1]:
from GeoParser import *
import pandas as pd
#pd.set_option("display.max_rows", None, "display.max_columns", None)

[WDM] - Downloading: 100%|██████████| 6.46M/6.46M [00:00<00:00, 39.2MB/s]


In [8]:
data=pd.read_csv("setGEF.csv")
data["geodeticDatum"]="(WGS84) World Geodetic System 1984"
#data["decimalLongitude"]=pd.NA
data["dynamicProperties"]=str({"textFieldMeasurementError": 10,"ChoiceDistUnits": "m", "ChoiceCoordSource": "GPS", "TextFieldExtent": 0})
#data.verbatimLatitude=data.verbatimLatitude.replace('"S"','" S')
#data.verbatimLongitude=data.verbatimLongitude.replace('O"',' "O')
#data.verbatimLongitude=data.verbatimLongitude.replace('W"',' "W')
data.head(2)

,Unnamed: 0,catalogNumber,verbatimLatitude,verbatimLongitude,verbatimCoordinates,verbatimSRS,decimalLongitude,decimalLatitude,geodeticDatum,dynamicProperties
0,1,GC01,3955218.07,513418.03,"19F 513418,03 3955218,07",WGS84,-68.792553,-54.550399,(WGS84) World Geodetic System 1984,"{'textFieldMeasurementError': 10, 'ChoiceDistU..."
1,2,GC02,3955146.47,513371.14,"19F 513371,14 3955146,47",WGS84,-68.793275,-54.551044,(WGS84) World Geodetic System 1984,"{'textFieldMeasurementError': 10, 'ChoiceDistU..."


In [9]:
%%time
#Testing...
DBPrep(data).verbatim_coordinates_autocomplete()
DBPrep(data).verbatim_coordinate_system_autocomplete()
DBPrep(data).verbatim_latitude_longitude_autocomplete()
#DBPrep(data).coordinate_precision_autocomplete() Deprecated function implemented on GeorefCalculator

CPU times: total: 3.12 s
Wall time: 3.34 s


In [10]:
data_pep=data.query("decimalLatitude.notna() and decimalLongitude.notna() ")
for index in data_pep.index:
    print(f"register {index} of {len(data_pep)}")
    verbatimLatitude=data_pep.loc[index,"decimalLatitude"]
    verbatimLongitude=data_pep.loc[index,"decimalLongitude"]
    dynamicProperties=data_pep.loc[index,"dynamicProperties"]
    verbatimCoordinteSystem="decimal degrees"
    verbatimSRS=data_pep.loc[index,"geodeticDatum"]
    SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).georeferencer()
    result_dict=SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).get_results()
    for keys in result_dict.keys():
        data.loc[index,keys]=result_dict[keys]
    

register 0 of 33376
register 1 of 33376
register 2 of 33376
register 3 of 33376
register 4 of 33376
register 5 of 33376
register 6 of 33376
register 7 of 33376
register 8 of 33376
register 9 of 33376
register 10 of 33376
register 11 of 33376
register 12 of 33376
register 13 of 33376
register 14 of 33376
register 15 of 33376
register 16 of 33376
register 17 of 33376
register 18 of 33376
register 19 of 33376
register 20 of 33376
register 21 of 33376
register 22 of 33376
register 23 of 33376
register 24 of 33376
register 25 of 33376
register 26 of 33376
register 27 of 33376
register 28 of 33376
register 29 of 33376
register 30 of 33376
register 31 of 33376
register 32 of 33376
register 33 of 33376
register 34 of 33376
register 35 of 33376
register 36 of 33376
register 37 of 33376
register 38 of 33376
register 39 of 33376
register 40 of 33376
register 41 of 33376
register 42 of 33376
register 43 of 33376
register 44 of 33376
register 45 of 33376
register 46 of 33376
register 47 of 33376
re

KeyboardInterrupt: 

In [11]:
result_dict

{'decimalLatitude': '-53.5028944',
 'decimalLongitude': '-69.1437083',
 'coordinateUncertaintyInMeters': '10',
 'geodeticDatum': '(WGS84) World Geodetic System 1984',
 'coordinatePrecision': '0.0000001',
 'georeferencedDate': '2022-11-16T06:19:41.148Z',
 'georeferenceProtocol ': 'Georeferencing Quick Reference Guide (Zermoglio et al. 2020, https://doi.org/10.35035/e09p-h128)'}

In [16]:
data_pep=data.query("verbatimLatitude.notna() and verbatimLongitude.notna() ")

for index in data_pep.index:
    print(f"register {index} of {len(data_pep)}")
    verbatimLatitude=data_pep.loc[index,"verbatimLatitude"]
    verbatimLongitude=data_pep.loc[index,"verbatimLongitude"]
    dynamicProperties=data_pep.loc[index,"dynamicProperties"]
    verbatimCoordinteSystem=data_pep.loc[index,"verbatimCoordinateSystem"]
    verbatimSRS=data_pep.loc[index,"verbatimSRS"]
    SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).georeferencer()
    result_dict=SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).get_results()
    for keys in result_dict.keys():
        data.loc[index,keys]=result_dict[keys]
driver.quit()

register 0 of 1645
register 1 of 1645
register 2 of 1645
register 3 of 1645
register 4 of 1645
register 5 of 1645
register 6 of 1645
register 7 of 1645
register 8 of 1645
register 9 of 1645
register 10 of 1645
register 11 of 1645
register 12 of 1645
register 13 of 1645
register 14 of 1645
register 15 of 1645
register 16 of 1645
register 17 of 1645
register 18 of 1645
register 19 of 1645
register 20 of 1645
register 21 of 1645
register 22 of 1645
register 23 of 1645
register 24 of 1645
register 25 of 1645
register 26 of 1645
register 27 of 1645
register 28 of 1645
register 29 of 1645
register 30 of 1645
register 31 of 1645
register 32 of 1645
register 33 of 1645
register 34 of 1645
register 35 of 1645
register 36 of 1645
register 37 of 1645
register 38 of 1645
register 39 of 1645
register 40 of 1645
register 41 of 1645
register 42 of 1645
register 43 of 1645
register 44 of 1645
register 45 of 1645
register 46 of 1645
register 47 of 1645
register 48 of 1645
register 49 of 1645
register 5

In [19]:
data.to_csv("resultadosGeorrefCEA.csv")

In [ ]:
%%time
# Testing ...
#coord_transform(data).utm_to_dg() Deprecated function implemented on GeorefCalculator
#CoordTransform(data).dms_to_dg() Deprecated function implemented on GeorefCalculator
#CoordTransform(data).geodetic_datum_autocomplete()  Deprecated function implemented on GeorefCalculator
CoordTransform(data).location_autocomplete() 
CoordTransform(data).continent_autocomplete() 
CoordTransform(data).higher_geography_autocomplete()

In [ ]:
list1=data.query('minimumElevationInMeters.isnull() and maximumElevationInMeters.isnull() and decimalLatitude.notnull() and decimalLongitude.notnull()').index.tolist()

In [ ]:
CoordTransform(data).elevation_autocomplete() 

In [ ]:
list2=data.query('minimumElevationInMeters.notnull() and maximumElevationInMeters.notnull() and decimalLatitude.notnull() and decimalLongitude.notnull()').index.tolist()

In [ ]:
list(set(list1) - set(list2))

In [12]:
data.to_csv()

,Unnamed: 0,...1,kingdom,class,family,scientificName,establishmentMeans,organismQuantity,basisOfRecord,Fuente del dato,...,decimalLongitude,dynamicProperties,verbatimSRS,verbatimCoordinates,verbatimCoordinateSystem,coordinateUncertaintyInMeters,geodeticDatum,coordinatePrecision,georeferencedDate,georeferenceProtocol
0,1,1,Animalia,Amphibia,Calyptocephalellidae,Calyptocephalella gayi,indigenous,1,Búsqueda intensiva,"Muñoz-Pedreros, Norambuena et al. (2013)",...,-72.2258333,"{'textFieldMeasurementError': 100, 'ChoiceDist...",,"40° 17' 26""S, 72° 13' 33""",degrees minutes seconds,2917,datum not recorded,0.0000001,2022-11-04T20:23:42.185Z,Georeferencing Quick Reference Guide (Zermogli...
1,2,2,Animalia,Amphibia,Ceratophryidae,Batrachyla leptopus,NaN,1,Búsqueda intensiva,"Muñoz-Pedreros, Norambuena et al. (2013)",...,-72.8680556,"{'textFieldMeasurementError': 100, 'ChoiceDist...",,"39°13'51""S, 72°52'05""O",degrees minutes seconds,2720,datum not recorded,0.0000001,2022-11-04T20:23:43.055Z,Georeferencing Quick Reference Guide (Zermogli...
2,3,3,Animalia,Amphibia,Ceratophryidae,Batrachyla leptopus,NaN,1,Búsqueda intensiva,"Muñoz-Pedreros, Norambuena et al. (2013)",...,-72.575,"{'textFieldMeasurementError': 100, 'ChoiceDist...",,"39°27'54""S, 72°34'30""O",degrees minutes seconds,2720,datum not recorded,0.0000001,2022-11-04T20:23:43.989Z,Georeferencing Quick Reference Guide (Zermogli...
3,4,4,Animalia,Amphibia,Ceratophryidae,Batrachyla leptopus,NaN,1,Búsqueda intensiva,"Muñoz-Pedreros, Norambuena et al. (2013)",...,-72.2258333,"{'textFieldMeasurementError': 100, 'ChoiceDist...",,"40° 17' 26""S, 72° 13' 33""",degrees minutes seconds,2917,datum not recorded,0.0000001,2022-11-04T20:23:44.879Z,Georeferencing Quick Reference Guide (Zermogli...
4,5,5,Animalia,Amphibia,Ceratophryidae,Batrachyla leptopus,NaN,1,Búsqueda intensiva,"Muñoz-Pedreros, Norambuena et al. (2013)",...,-72.8677778,"{'textFieldMeasurementError': 100, 'ChoiceDist...",,"43°38'01"" S, 72°52'04"" O",degrees minutes seconds,2767,datum not recorded,0.0000001,2022-11-04T20:23:45.647Z,Georeferencing Quick Reference Guide (Zermogli...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,1647,1647,Animalia,Reptilia,Tropiduridae,Microlophus tarapacensis,NaN,30,Busqueda Intensiva,"Muñoz-Pedreros, Norambuena et al. (2013)",...,-70.3322222,"{'textFieldMeasurementError': 100, 'ChoiceDist...",,"18°28'49""S, 70°19'56""W",degrees minutes seconds,2820,datum not recorded,0.0000001,2022-11-04T20:57:31.382Z,Georeferencing Quick Reference Guide (Zermogli...
1647,1648,1648,Animalia,Reptilia,Tropiduridae,Microlophus tarapacensis,NaN,8,Busqueda Intensiva,"Muñoz-Pedreros, Norambuena et al. (2013)",...,-70.3261111,"{'textFieldMeasurementError': 100, 'ChoiceDist...",,"18°29'42""S, 70°19'34""W",degrees minutes seconds,2820,datum not recorded,0.0000001,2022-11-04T20:57:32.952Z,Georeferencing Quick Reference Guide (Zermogli...
1648,1649,1649,Animalia,Reptilia,Tropiduridae,Microlophus tarapacensis,NaN,2,Busqueda Intensiva,"Muñoz-Pedreros, Norambuena et al. (2013)",...,-70.3311111,"{'textFieldMeasurementError': 100, 'ChoiceDist...",,"18°33'07""S, 70°19'52""W",degrees minutes seconds,2820,datum not recorded,0.0000001,2022-11-04T20:57:34.530Z,Georeferencing Quick Reference Guide (Zermogli...
1649,1650,1650,Animalia,Reptilia,Tropiduridae,Microlophus tarapacensis,NaN,3,Busqueda Intensiva,"Muñoz-Pedreros, Norambuena et al. (2013)",...,<NA>,"{'textFieldMeasurementError': 100, 'ChoiceDist...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN
